## Chapter 6
# Resonance

## The Derivative

The derivative of a function $f(x) = y$ with respect to the dependent variable $x$ is defined as:

$\frac{dy}{dx} = \lim_\limits{\Delta x \to 0}\frac{f(x + \Delta x) - f(x)}{\Delta x}$

In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipython_animation import create_animation, DEFAULT_FPS
%matplotlib inline

# `f` must accepts a single numpy array
def animate_derivative_limit(f, actual_derivative_f, function_name='$f(x)$', x_range=np.linspace(-4 * np.pi, 4 * np.pi, 100), ylim=(-1.1, 1.1), animation_length_seconds=3):
    max_delta_x = 5.0
    num_frames = animation_length_seconds * DEFAULT_FPS
    
    fig = plt.figure(figsize=(8, 4))

    ax, = plt.plot(x_range, np.zeros(len(x_range)), label='Est. derivative')
    plt.plot(x_range, actual_derivative_f(x_range), label='Actual derivative', alpha=0.6)
    plt.xlabel('$x$')
    plt.ylabel(function_name)
    plt.ylim(ylim)
    plt.legend(loc='lower left')

    def animate(frame):
        delta_x = max_delta_x * (num_frames - frame) / num_frames
        values = (f(x_range + delta_x) - f(x_range)) / delta_x
        ax.set_ydata(values)
        plt.title('Estimating derivative of ' + function_name + ' w/ $\\Delta x = %0.3f$' % delta_x)
    return create_animation(fig, plt, animate, length_seconds=animation_length_seconds)

### Derivative of the Sine Function

$\frac{d}{d\theta}\sin \theta = \cos \theta$

In [3]:
animate_derivative_limit(np.sin, np.cos, function_name='$\sin(x)$')

### Derivative of the Cosine Function

$\frac{d}{d\theta}\cos \theta = -\sin \theta$

In [4]:
animate_derivative_limit(np.cos, lambda x: -np.sin(x), function_name='$\cos(x)$')

### Derivative of $e$

The derivative of $e^x$ is identical to $e^x$:

$\frac{d}{dx}e^{ax} = ae^{ax}$

In [5]:
animate_derivative_limit(np.exp, np.exp, function_name='$e^{x}$', x_range=np.linspace(-8, 4, 100), ylim=(-0.5, 10))

$\frac{d}{dx}\ln{ax} = \frac{1}{x}$

In [6]:
animate_derivative_limit(np.log, lambda x: 1/x, function_name='$\\ln{x}$', x_range=np.linspace(1e-8, 4, 100), ylim=(-0.5, 10))

**Chain Rule:** **_The derivative of $f(g(x))$ is the derivative of $f$ with respect to $g(x)$ times the derivative of $g(x)$._**

For example,
$\frac{d}{dx}\ln{\sin x} = \frac{1}{\sin x} \cdot \frac{d}{dx}\sin x = \frac{\cos x}{\sin x}$:

In [7]:
animate_derivative_limit(lambda x: np.log(np.sin(x)), lambda x: np.cos(x)/np.sin(x), function_name='$\\ln(\\sin{(x)})$', x_range=np.linspace(1e-8, np.pi, 100), ylim=(-10, 10))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


## Damped Harmonic Oscillator

Displacement $x$ of a damped harmonic oscillator:

$x(t) = e^{\omega t} = e^{-\frac{c}{2m}t}e^{-\frac{\sqrt{c^2-4km}}{2m}t}$,

where $m$ is mass of the system, $c$ is a constant called the _damping factor_, $k$ is the elastic restoring force constant, or spring constant, $\omega$ is the angular velocity, and $t$ is time.

Note that in the second term, if $c^2-4km$ is positive, then the second term is real, and so it will not vibrate and sill instead be a simple exponential decay function.  If it is negative, however, the exponent will be imaginary and the second term will be a phasor that vibrates.  The system does not vibrate when dissipation dominates over other forces.

In [8]:
def dho_displacement(m, c, k, t):
    return np.exp(-c*t/(2 * m)) * np.exp(-np.sqrt(c ** 2 - 4 * k * m + 0j) * t / (2 * m)).real

In [9]:
m = 2
t_max = 200
t_range = np.arange(t_max)
fig = plt.figure(figsize=(10, 12))
fig.suptitle('Damped harmonic oscillator', size=16)

m_plot = plt.subplot(312)
c_plot = plt.subplot(311)
k_plot = plt.subplot(313)

m_ax, = m_plot.plot(t_range, np.zeros(t_max), linewidth=3)
c_ax, = c_plot.plot(t_range, np.zeros(t_max), linewidth=3)
k_ax, = k_plot.plot(t_range, np.zeros(t_max), linewidth=3)

for plot in [m_plot, c_plot, k_plot]:
    plot.set_title('Temp') # needed for `tight_layout()` to work correctly before animation
    plot.axis([0, t_max, -1.1, 1.1])
    plot.set_xlabel('Time')
    plot.set_ylabel('Displacement')

animation_length_seconds = 4
num_frames = animation_length_seconds * DEFAULT_FPS

m_range = np.logspace(-1, 2, num_frames)
c_range = np.logspace(-4, 1, num_frames)
k_range = np.logspace(-3, 0, num_frames)

default_m = 2
default_c = 0.1
default_k = 0.1

def animate(frame):
    m = m_range[frame]
    m_plot.set_title('Mass $m = %0.3f$' % m, size=12)
    m_ax.set_ydata(dho_displacement(m, default_c, default_k, t_range))

    c = c_range[frame]
    c_plot.set_title('Damping factor $c = %0.3f$' % c, size=12)
    c_ax.set_ydata(dho_displacement(default_m, c, default_k, t_range))

    k = k_range[frame]
    k_plot.set_title('Spring constant $k = %0.3f$' % k, size=12)
    k_ax.set_ydata(dho_displacement(default_m, default_c, k, t_range))

plt.tight_layout()
plt.subplots_adjust(top=0.90) # make room for suptitle

create_animation(fig, plt, animate, length_seconds=animation_length_seconds)

## Driven Harmonic Oscillator

The magnitude of the response of a driven harmonic oscillator with a sinusoidal driving function with varying driving frequency $\omega$ is given by

$\left|y(t)\right| = \frac{F}{\sqrt{(k - \omega^2m)^2 + \omega^2 c^2}}$,

where the driving funcion has amplitude $F$ and angular velocity $\omega$. $k$, $m$, and $c$ represent the same things as in the _damped harmonic oscillator_ above (spring constant, system mass and damping factor, respectively).

The resonant frequency (the frequency of the driving function) producing the maximum response magnitude) can be found with

$\omega_{rdd} = \pm \sqrt{\frac{k}{m} - \frac{c^2}{2m^2}}$

In [10]:
def dho_response_magnitude(driving_frequency, driving_amplitude, m, c, k):
    w = driving_frequency
    return driving_amplitude / np.sqrt((k - w ** 2 * m) ** 2 + w ** 2 * c ** 2)

In [11]:
def dho_resonant_frequency(m, c, k):
    return np.sqrt(k / m - c ** 2 / (2 * m ** 2))

The following animation emulates _Figure 6.18_ on _p297_.

In [12]:
m = 3 # kg
c = 7 # Ns/m
k = 60 # N/m
F = 1 # N

driving_frequency_range = np.linspace(0, 5 * np.pi, 100)

fig = plt.figure(figsize=(10, 8))
plt.xlabel('Frequency')
plt.ylabel('Magnitude')

ax, = plt.plot(driving_frequency_range, np.zeros(len(driving_frequency_range)), linewidth=3, label='Response magnitude')
rf_line, = plt.plot([0, 0], [0, 1], color='red', linewidth=1, label='Resonant frequency')
plt.ylim(0, 0.2)
plt.legend()

c_range = np.logspace(0.1, 1, 100)

def animate(frame):
    c = c_range[frame]
    ax.set_ydata(dho_response_magnitude(driving_frequency_range, F, m, c, k))
    rf_x = dho_resonant_frequency(m, c, k)
    rf_line.set_xdata(rf_x)
    plt.title('Response spectrum of a driven harmonic oscillator\nwith damping factor $c = %0.3f$' % c, size=14)
create_animation(fig, plt, animate, length_seconds=animation_length_seconds)